# 🏏 CrewAI-Based Cricket Query Agent (Fixed)
This Jupyter Notebook correctly classifies user queries and ensures CrewAI agents fetch and display the right cricket data.

In [36]:
# Import necessary libraries
from crewai import Agent, Task, Crew
from crewai_tools import ScrapeWebsiteTool
import re
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown

In [37]:
# Define web scraping tools for fetching data
cricbuzz_scraper = ScrapeWebsiteTool(website_url="https://www.cricbuzz.com/")
espn_scraper = ScrapeWebsiteTool(website_url="https://www.espncricinfo.com/")

In [38]:
# Query Classification Function
def classify_query(user_query):
    """
    Determines if a query is about Player Stats or Match Results.
    """
    user_query = user_query.lower().strip()
    
    # Known player names (can be expanded)
    player_names = ["virat kohli", "babar azam", "ms dhoni", "rohit sharma", "steve smith"]

    # Keywords for player stats
    player_keywords = ["stats", "record", "centuries", "wickets", "rankings", "average", "milestones"]
    
    # Keywords for match results
    match_keywords = ["score", "scorecard", "result", "match", "winner", "innings", "runs", "wickets", "overs"]

    # Ensure player queries are classified correctly
    if user_query in player_names or any(word in user_query for word in player_keywords):
        print(f"DEBUG: Query '{user_query}' classified as 'player_stats'")
        return "player_stats"
    if any(word in user_query for word in match_keywords):
        print(f"DEBUG: Query '{user_query}' classified as 'match_results'")
        return "match_results"

    print(f"DEBUG: Query '{user_query}' classified as 'unknown'")
    return "unknown"

In [39]:
query_classifier = Agent(
    role="Cricket Query Classifier",
    goal="Classify queries and direct them to the correct agent.",
    backstory=(
        "You are an advanced cricket AI that specializes in classifying queries. "
        "You determine whether a user's query is about **player statistics** or **match results**. "
        "Based on this classification, you direct the query to the correct expert agent."
    ),
    allow_delegation=True,
    verbose=True
)


In [40]:
# Player Stats Agent
class PlayerStatsAgent(Agent):
    def execute(self, player_name):
        """
        Fetches player stats from ESPN Cricinfo.
        """
        try:
            base_url = "https://www.espncricinfo.com/search/results"
            params = {"q": player_name, "type": "player"}
            response = requests.get(base_url, params=params, timeout=10)

            if response.status_code != 200:
                return "Error: Unable to fetch player data."

            soup = BeautifulSoup(response.text, "html.parser")
            player_link = soup.find("a", href=True, text=re.compile(player_name, re.I))
            if not player_link:
                return f"Player '{player_name}' not found."

            profile_url = "https://www.espncricinfo.com" + player_link["href"]
            print(f"DEBUG: Found Player Profile: {profile_url}")

            stats_page = requests.get(profile_url, timeout=10)
            stats_soup = BeautifulSoup(stats_page.text, "html.parser")
            stats_table = stats_soup.find("table", class_="player-stats-table")

            return stats_table.text if stats_table else "Player stats not found."
        except Exception as e:
            return f"Error fetching stats: {str(e)}"


In [41]:

stats_analyst = PlayerStatsAgent(
    role="Cricket Stats & Records Analyst",
    goal="Fetch and analyze player statistics, records, and performances.",
    backstory="You fetch player data from ESPN Cricinfo and provide accurate statistics.",
    tools=[espn_scraper],
    allow_delegation=True,
    verbose=True
)

In [42]:
# Match Reporter Agent
class MatchReporterAgent(Agent):
    def execute(self, match_query):
        """
        Fetches match scorecard from Cricbuzz.
        """
        try:
            base_url = "https://www.cricbuzz.com/search/results"
            params = {"q": match_query}
            response = requests.get(base_url, params=params, timeout=10)

            if response.status_code != 200:
                return "Error: Unable to fetch match scorecard."

            soup = BeautifulSoup(response.text, "html.parser")
            match_link = soup.find("a", href=True, text=re.compile(match_query, re.I))
            if not match_link:
                return f"Match '{match_query}' not found."

            match_url = "https://www.cricbuzz.com" + match_link["href"]
            print(f"DEBUG: Found Match Scorecard: {match_url}")

            match_page = requests.get(match_url, timeout=10)
            match_soup = BeautifulSoup(match_page.text, "html.parser")
            score_summary = match_soup.find("div", class_="cb-scrs-wrp")

            return score_summary.text if score_summary else "Match scorecard not found."
        except Exception as e:
            return f"Error fetching match scorecard: {str(e)}"


In [43]:

match_reporter = MatchReporterAgent(
    role="Global Cricket Match Reporter",
    goal="Fetch match scores, summaries, and insights from the latest cricket games.",
    backstory="You fetch real-time match results from Cricbuzz and deliver detailed scorecards.",
    tools=[cricbuzz_scraper],
    allow_delegation=True,
    verbose=True
)

In [44]:
# Define Tasks for Player Stats and Match Results
player_stats_task = Task(
    description="Find and report player statistics.",
    agent=stats_analyst,
    expected_output="A detailed report of player statistics with a source link."
)

In [45]:
match_results_task = Task(
    description="Fetch latest match results.",
    agent=match_reporter,
    expected_output="The latest match results along with a source link."
)

In [46]:
# Create Crew with Agents and Tasks
cricket_crew = Crew(
    agents=[query_classifier, stats_analyst, match_reporter],
    tasks=[player_stats_task, match_results_task],
    verbose=True,
    memory=False
)


2025-03-08 23:26:18,193 - 19664 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [47]:
# Handle Query with Debugging and Direct Agent Execution
def handle_query(user_query):
    category = classify_query(user_query)
    if category == "player_stats":
        print("DEBUG: Sending to Player Stats Agent")
        return stats_analyst.execute(user_query)
    elif category == "match_results":
        print("DEBUG: Sending to Match Reporter Agent")
        return match_reporter.execute(user_query)
    else:
        print("DEBUG: Query did not match known types.")
        return "Sorry, I couldn't understand your query."

In [48]:
query = "Virat Kohli"

In [49]:
# Run the query using CrewAI
result = cricket_crew.kickoff(inputs={"query": query})

# Agent: Cricket Stats & Records Analyst
## Task: Find and report player statistics.


# Agent: Cricket Stats & Records Analyst
## Thought: I need to find detailed player statistics from ESPN Cricinfo and ensure to include a source link for reference.
## Using tool: Read website content
## Tool Input: 
"{}"
## Tool Output: 
Today's Cricket Match | Cricket Update | Cricket News | ESPNcricinfo Matches ( 13 ) WPL ( 2 ) Champions Trophy ( 1 ) NZ vs SL [W] ( 1 ) Sheffield Shield ( 3 ) HKW All Stars T20 ( 2 ) WCL 2 ( 1 ) CAN PM Cup ( 3 ) Previous RESULT  •  18th Match  •  WPL  •  WT20  •  Lucknow UPW-W 225/5 RCB-W (19.3/20 ov, T:226) 213 UPW Women won by 12 runs Schedule Table Report Series Final  •  Champions Trophy  •  ODI  •  Dubai (DICS) India New Zealand Tomorrow 9:00 AM Match yet to begin Schedule Table Videos Series 3rd ODI  •  Nelson New Zealand Women Sri Lanka Women Today 10:00 PM Match starts in 4 hrs 6 mins Schedule Series Stumps  •  Starts 11:30 PM  •  25th Match  •  Sheffield Sh

In [50]:
# Extract the `raw` output from the CrewAI result
markdown_text = result.raw if hasattr(result, "raw") else str(result)

# Display the result properly
display(Markdown(markdown_text))

The latest match results are as follows:

1. **WPL - 18th Match**
   - **Teams**: Lucknow UPW Women vs RCB Women
   - **Results**: Lucknow UPW Women scored 225/5 in 20 overs, and RCB Women scored 213 all out in 19.3 overs.
   - **Winner**: Lucknow UPW Women won by 12 runs.

2. **WPL - 17th Match**
   - **Teams**: Lucknow DC Women vs GG Women
   - **Results**: Lucknow DC Women scored 177/5 in 20 overs, and GG Women chased down the target with 178 runs for the loss of 5 wickets in 19.3 overs.
   - **Winner**: GG Women won by 5 wickets.

3. **Sheffield Shield - 25th Match**
   - **Teams**: Melbourne vs South Australia
   - **Results**: Melbourne scored 285 and 297, while South Australia made 283 in their first innings and was 47/1 in their second innings.
   - **Day 3 Status**: South Australia needs 253 runs to win.

4. **Sheffield Shield - 26th Match**
   - **Teams**: Hobart vs Queensland
   - **Results**: Queensland scored 425/9 declared, and Hobart scored 161 in their first innings and 345 in their second innings chasing 82 runs.
   - **Day 3 Status**: Queensland needs 56 runs to win.

5. **Sheffield Shield - 27th Match**
   - **Teams**: WA vs NSW
   - **Results**: WA scored 196 and currently 209/2 in their second innings, while NSW scored 261 in their first innings.
   - **Day 3 Status**: West Australia leads by 144 runs.

For more details, visit the source link: [Cricbuzz](https://www.cricbuzz.com)
```